In [ ]:
import sys
import os
sys.path.append("../../TychePlot/")
from OLEDPlot import OLEDPlot
from Filereader import fileToNpArray,npArrayToFile
import numpy as np

In [ ]:
vial_calib="calib/SH_vial_calib"
thinfilm_calib="calib/SH_thinfilm_calib"
sphere_calib="calib/SH_sphere_calib"

In [ ]:
subdirnames=['perovskites_solution', 'perovskites_sphere', 'Thinfilm_perovskites']

In [ ]:
file_format={
                "skiplines":1,
                "fileEnding":".tsv",
                "separator":"\t",
                "commaToPoint":True,
            }
clbr_format={
                "skiplines":1,
                "fileEnding":".clbr",
                "separator":"\t",
            }

In [ ]:
files=[[os.path.join(subdir,file[:-4]) for file in os.listdir(subdir)] for subdir in subdirnames]
files

In [ ]:
c=2.99793*10**8
h=6.623*10**-34
hc=h*c

In [ ]:
calc_PLQE_with=[
                (0,'Laser','Sample','SamplePassive'),
                ('Laser','SampleLongExp','GlassLongExp','SamplePassiveLongExp','GlassPassiveLongExp','Sample','SamplePassive','GlassPassive','Glass',0),
                ('GlassLongExp','SamplePassiveLongExp','Sample','SamplePassive','Laser',0,'GlassPassiveLongExp','Glass','GlassPassive',0,'SampleLongExp'),
                ]

In [ ]:
files_exp_rep=[[{"file":file,"exp_time":file.split("_")[-4][:-1],"rep":file.split("_")[-5][:-3]} for file in files_sub] for files_sub in files]

In [ ]:
for files_sub in files_exp_rep:
    for file in files_sub:
        file['rep']=int(file['rep'])
        try:
            file['exp_time']=int(file['exp_time'])
        except ValueError as e:
            if str(e)[-2] =='m':
                file['exp_time']=int(file['exp_time'][:-1])/1000
         #file['exp_time']=

In [ ]:
PLQE_inputs=[{key:value for key,value in zip(calc_PLQE_sub,files_sub) if key != 0} for calc_PLQE_sub,files_sub in zip(calc_PLQE_with,files_exp_rep)]

In [ ]:
spectra_offset=[350,-100]
laser_thresh=420*10**-9

In [ ]:
wavelengths=fileToNpArray(vial_calib, **clbr_format)[0][:,0][spectra_offset[0]:spectra_offset[1]]
laser_thresh_index=np.argwhere(wavelengths > laser_thresh)[0][0]
calibs={
            'vial_calib_vec':fileToNpArray(vial_calib, **clbr_format)[0][:,1][spectra_offset[0]:spectra_offset[1]],
            'thinfilm_calib_vec':fileToNpArray(thinfilm_calib, **clbr_format)[0][:,1][spectra_offset[0]:spectra_offset[1]],
            'sphere_calib_vec':fileToNpArray(sphere_calib, **clbr_format)[0][:,1][spectra_offset[0]:spectra_offset[1]]
}
calibs_input=['vial_calib_vec','sphere_calib_vec','thinfilm_calib_vec']

In [ ]:
for PLQE_input,calib in zip(PLQE_inputs,calibs_input):
    PLQE_input['calib']=calibs[calib]
    for value in PLQE_input.values():
        try:
            value['spectra']=fileToNpArray(value['file'], **file_format)[0][:,1][spectra_offset[0]:spectra_offset[1]]
            value['spectra']/=value['exp_time']*value['rep']
            value['spectra']*=PLQE_input['calib']
        except IndexError:
            pass
    try: 
        PLQE_input['Sample']['spectra']=np.hstack((PLQE_input['Sample']['spectra'][:laser_thresh_index],PLQE_input['SampleLongExp']['spectra'][laser_thresh_index:]))
    except KeyError:
        pass
    try:
        PLQE_input['SamplePassive']['spectra']=np.hstack((PLQE_input['SamplePassive']['spectra'][:laser_thresh_index],PLQE_input['SamplePassiveLongExp']['spectra'][laser_thresh_index:]))
    except KeyError:
        pass

In [ ]:
import scipy.interpolate as sci
import scipy.integrate as scin
import matplotlib.pyplot as plt
n=0
for PLQE_input in PLQE_inputs:
    plt.figure(n)
    plt.plot(wavelengths, PLQE_input['Laser']['spectra'], wavelengths, PLQE_input['Sample']['spectra'], wavelengths, PLQE_input['SamplePassive']['spectra'])
    #plt.ylim(10**-9,10**-4)
    plt.yscale("log")
    n+=1

In [ ]:
### PAPER CALCS

In [ ]:
## Summation
delta_x=[]
for x in range(0,len(wavelengths)-1):
    delta_x.append(wavelengths[x+1]-wavelengths[x])
L_a_midpoint_riemann_sum_elements=[]
L_b_midpoint_riemann_sum_elements=[]
L_c_midpoint_riemann_sum_elements=[]
for PLQE_dict in PLQE_inputs:
    laser=PLQE_dict['Laser']['spectra']
    sample=PLQE_dict['Sample']['spectra']
    laser_sample=PLQE_dict['SamplePassive']['spectra']
    for x in range(0,len(delta_x)):
        L_a_midpoint_riemann_sum_elements.append(((laser[x]+laser[x+1])/2)*10**9*((wavelengths[x]+wavelengths[x+1])/2)/hc*delta_x[x])
        L_b_midpoint_riemann_sum_elements.append(((laser_sample[x]+laser_sample[x+1])/2)*10**9*((wavelengths[x]+wavelengths[x+1])/2)/hc*delta_x[x])
        L_c_midpoint_riemann_sum_elements.append(((sample[x]+sample[x+1])/2)*10**9*((wavelengths[x]+wavelengths[x+1])/2)/hc*delta_x[x])
    L_a_sum=sum(L_a_midpoint_riemann_sum_elements[:laser_thresh_index])
    L_b_sum=sum(L_b_midpoint_riemann_sum_elements[:laser_thresh_index])
    L_c_sum=sum(L_c_midpoint_riemann_sum_elements[:laser_thresh_index])
    P_a_sum=sum(L_a_midpoint_riemann_sum_elements[laser_thresh_index:])
    P_b_sum=sum(L_b_midpoint_riemann_sum_elements[laser_thresh_index:])
    P_c_sum=sum(L_c_midpoint_riemann_sum_elements[laser_thresh_index:])
    # 2 Step
    PLQY_2_step_sum=(P_c_sum)/(L_a_sum-L_c_sum)*100
    # 3 Step
    PLQY_3_step_sum=(P_c_sum-(L_c_sum/L_b_sum)*P_b_sum)/(L_a_sum*(1-L_c_sum/L_b_sum))*100
    print(
        f"\nPLQY by Summation:\n"+
        f"\nPLQY by 2 Step: {PLQY_2_step_sum:5.2f} %\nPLQY by 3 Step: {PLQY_3_step_sum:5.2f} %")

In [ ]:
for PLQE_dict in PLQE_inputs:
    laser=PLQE_dict['Laser']['spectra']
    sample=PLQE_dict['Sample']['spectra']
    laser_sample=PLQE_dict['SamplePassive']['spectra']
    ## Interpolation
    P_a_spectral_func=sci.interp1d(wavelengths*10**9,laser, kind='cubic')
    P_b_spectral_func=sci.interp1d(wavelengths*10**9,laser_sample, kind='cubic')
    P_c_spectral_func=sci.interp1d(wavelengths*10**9,sample, kind='cubic')
    # Integration
    L_a, L_a_err=scin.quad(lambda a: a*P_a_spectral_func(a)/hc,wavelengths[0],laser_thresh, limit=5000)
    L_b, L_b_err=scin.quad(lambda a: a*P_b_spectral_func(a)/hc,wavelengths[0],laser_thresh, limit=5000)
    L_c, L_c_err=scin.quad(lambda a: a*P_c_spectral_func(a)/hc,wavelengths[0],laser_thresh, limit=5000)
    P_a, P_a_err=scin.quad(lambda a: a*P_a_spectral_func(a)/hc,laser_thresh,wavelengths[-1], limit=5000)
    P_b, P_b_err=scin.quad(lambda a: a*P_b_spectral_func(a)/hc,laser_thresh,wavelengths[-1], limit=5000)
    P_c, P_c_err=scin.quad(lambda a: a*P_c_spectral_func(a)/hc,laser_thresh,wavelengths[-1], limit=5009)
    # 2 Step
    PLQY_2_step=(P_c)/(L_a-L_c)*100
    # 3 Step
    PLQY_3_step=(P_c-(L_c/L_b)*P_b)/(L_a*(1-L_c/L_b))*100
    print(
        f"\nPLQY by Integration:\n"+
        f"\nPLQY by 2 Step: {PLQY_2_step_sum:5.2f} %\nPLQY by 3 Step: {PLQY_3_step_sum:5.2f} %")